## Проект "Рекомендация тарифов"

В нашем распоряжении выборка данных о поведении клиентов, которые уже перешли на тарифные планы "Smart" и "Ultra". Нужно построить модель для задачи классификации, которая выберет подходящий тариф для клиентов которые еще находятся на архивных тарифных планах. Предобработка данных не понадобится — мы её уже сделали.

**Цель:** Построить модель категоризации с *accuracy* по крайней мере до 0.75. Проверить *accuracy* модели на тестовой выборке.

**Задачи:**

1. Открыть файл с данными и изучить его. Путь к файлу: /datasets/users_behavior.csv.
 
2. Разделить исходные данные на обучающую, валидационную и тестовую выборки.

3. Исследовать качество разных моделей, меняя гиперпараметры. Сделать выводы по исследованию.

4. Проверить качество модели на тестовой выборке.

5. Проверить модель на вменяемость.

**Предмет исследования:** выборка данных по пользователям оператора «Мегалайна»

**Методы** которые мы применим в данном исследовании включают в себя: ознакомление и проверку данных, библиотеку "sklearn", обучение и проверку моделей.

**Структура проекта:** Проектная работа состоит из обзора и предобработки данных, затем следует разбивка данных на выборки, иссследование моделей, проверка моделей, проверка адекватности модели и общий вывод.

## Откройте и изучите файл

In [1]:
## Импорт библиотек

import pandas as pd
import numpy as np
import collections, numpy
import matplotlib.pyplot as plt

from sklearn import tree
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

Откроем файл и запишем данные в пременную "df":

In [2]:
df = pd.read_csv('/Users/Daniil/Downloads/users_behavior.csv')

Выведем первые пять строк:

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Ознакомимся с общей информацией о датасете:

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Нулевых и пропушенных значений не обнаружено.

Преобразуем типы данных в стобцах в цельночисленные, с округлением минут и сообщений до целых чисел, мегабайт до гигабайт, и переименуем столбец 'mb_used' в 'gb_used'.

In [5]:
df['calls'] = df['calls'].astype('int')
df['minutes'] = np.ceil(df['minutes']).astype('int')
df['messages'] = df['messages'].astype('int')
df['mb_used'] = np.ceil(df['mb_used'] / 1024).astype('int')

df.rename(columns={'mb_used': 'gb_used'}, inplace=True)

df.head()

,calls,minutes,messages,gb_used,is_ultra
0,40,312,83,20,0
1,85,517,56,23,0
2,77,468,86,21,0
3,106,746,81,9,1
4,66,419,1,15,0


Проверим на дубликаты:

In [6]:
df.duplicated().sum()

2

Две строки дублированы

In [7]:
df[df.duplicated(keep=False)]


,calls,minutes,messages,gb_used,is_ultra
540,2,2,0,1,1
1065,2,2,0,1,1
1821,16,102,0,3,0
2396,16,102,0,3,0


Удалим их

In [8]:
df = df.drop_duplicates()
df.shape

(3212, 5)

С значениями тоже все впорядке:

In [9]:
df.describe()

,calls,minutes,messages,gb_used,is_ultra
count,3212.000000,3212.000000,3212.000000,3212.000000,3212.000000
mean,63.072540,438.939913,38.305106,17.309465,0.306351
std,33.218875,234.459477,36.146950,7.391705,0.461049
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,275.000000,9.000000,13.000000,0.000000
50%,62.000000,431.000000,30.000000,17.000000,0.000000
75%,82.000000,573.000000,57.000000,21.000000,1.000000
max,244.000000,1633.000000,224.000000,49.000000,1.000000


А вот с классами не все так радужно - в нашем датасете присутсвует разбалансированность классов:

In [10]:
print(f"Количество строк в df['is_ultra'] по классам: {np.bincount(df['is_ultra'])}")

Количество строк в df['is_ultra'] по классам: [2228  984]


## Разбейте данные на выборки

Выделим признаки и целевое значение:

In [11]:
features = df.drop('is_ultra', axis=1)  # features: содержит признаки
target = df['is_ultra']  # features: содержит целевой показатель

Разделим наш датасет на обучающую и тестовую - выборки:

In [12]:
features_train, features_test, target_train, target_test = train_test_split( # 60/40
    features, target, test_size=0.75, random_state=1234, stratify=target)

print(f"Количество строк в target_train по классам: {np.bincount(target_train)}")
print(f"Количество строк в target_test по классам: {np.bincount(target_test)}")

Количество строк в target_train по классам: [557 246]
Количество строк в target_test по классам: [1671  738]


## Исследуйте модели

Мы будем обучать три модели: **LogisticRegression**, **DecisionTreeClassifier**, **RandomForestClassifier** , для определения оптимальных парметров настройки моделей - используем **GridSearchCV**:

### LogisticRegression

In [13]:
%%time
scaler = StandardScaler() # Модель StandardScaler() поместим в переменную scaler
logistic = LogisticRegression() # Модель StandardScaler() поместим в переменную logistic

pipe = Pipeline(steps=[("scaler", scaler), ("logistic", logistic)]) # Создаем пайплайн с двумя шагами

 # Задаем решетку параметров пайплайна
param_grid = {'logistic__penalty': ['l1', 'l2'],
              'logistic__C': np.logspace(-2, 2, 20),
              'logistic__solver': ['liblinear'],
              'logistic__max_iter': list(range(400, 800, 100)),
              'logistic__random_state':[12345]}

 # Создаем нашу модель и передаем ей пайплайн, решетку параметров, также используем кросс-валидацию результата
grid_search_cv_lr = GridSearchCV(pipe, param_grid, cv=5)

 # Обучаем модель на тренировочной части выборки:
grid_search_cv_lr.fit(features_train, target_train)

best_params_lr = grid_search_cv_lr.best_params_ # Лучшие параметры
best_lr = grid_search_cv_lr.best_estimator_ # Лучшая модель
best_lr_score = grid_search_cv_lr.best_score_ # Лучший результат

print(f'Точность модели {logistic} на тренировочной выборке составила: \
    {best_lr_score} с параметрами \
    {best_params_lr}')

Точность модели LogisticRegression() на тренировочной выборке составила:     0.7471972049689442 с параметрами     {'logistic__C': 0.04281332398719394, 'logistic__max_iter': 400, 'logistic__penalty': 'l2', 'logistic__random_state': 12345, 'logistic__solver': 'liblinear'}
Wall time: 1.93 s


### DecisionTreeClassifier

In [14]:
%%time
dtc = tree.DecisionTreeClassifier() # Модель DecisionTreeClassifier() поместим в переменную dtc

pipe = Pipeline(steps=[("scaler", scaler), ("dtc", dtc)]) # Создаем пайплайн с двумя шагами

 # Задаем решетку параметров пайплайна
param_grid =  {'dtc__max_depth': list(range(8, 10)),
              'dtc__min_samples_split': list(range(20, 26)),
              'dtc__min_samples_leaf': list(range(1, 3)),
              'dtc__criterion': ['entropy', 'gini'],
              'dtc__splitter': ['random'],
             'dtc__random_state':[12345]}


 # Создаем нашу модель и передаем ей пайплайн, решетку параметров, также используем кросс-валидацию результата    
grid_search_cv_dtc = GridSearchCV(pipe, param_grid, n_jobs=8, cv=5)

 # Обучаем модель на тренировочной части выборки:
grid_search_cv_dtc.fit(features_train, target_train)

best_params_dtc = grid_search_cv_dtc.best_params_ # Лучшие параметры
best_dtc = grid_search_cv_dtc.best_estimator_ # Лучшая модель
best_dtc_score = grid_search_cv_dtc.best_score_ # Лучший результат

print(f'Точность модели {dtc} на тренировочной выборке составила: \
    {best_dtc_score} с параметрами \
    {best_params_dtc}')

Точность модели DecisionTreeClassifier() на тренировочной выборке составила:     0.7820729813664596 с параметрами     {'dtc__criterion': 'gini', 'dtc__max_depth': 8, 'dtc__min_samples_leaf': 2, 'dtc__min_samples_split': 20, 'dtc__random_state': 12345, 'dtc__splitter': 'random'}
Wall time: 775 ms


### RandomForestClassifier

In [15]:
%%time

rfc = RandomForestClassifier() # Модель RandomForestClassifier() поместим в переменную rfc

pipe = Pipeline(steps=[("scaler", scaler), ("rfc", rfc)]) # Создаем пайплайн с двумя шагами

 # Задаем решетку параметров пайплайна
param_grid =  {'rfc__n_estimators': list(range(10, 15, 1)), 
              'rfc__max_depth': list(range(6, 10)), 
              'rfc__bootstrap': [True, False], 
              'rfc__criterion': ['gini', 'entropy'],
             'rfc__random_state':[12345]}


 # Создаем нашу модель и передаем ей пайплайн, решетку параметров, также используем кросс-валидацию результата    
grid_search_cv_rfc = GridSearchCV(pipe, param_grid, n_jobs=8, cv=5)

 # Обучаем модель на тренировочной части выборки:
grid_search_cv_rfc.fit(features_train, target_train)

best_params_rfc = grid_search_cv_rfc.best_params_ # Лучшие параметры
best_rfc = grid_search_cv_rfc.best_estimator_ # Лучшая модель
best_rfc_score = grid_search_cv_rfc.best_score_ # Лучший результат
print(f'Точность модели {rfc} на тренировочной выборке составила: \
    {best_rfc_score} с параметрами \
    {best_params_rfc}')

Точность модели RandomForestClassifier() на тренировочной выборке составила:     0.8057065217391305 с параметрами     {'rfc__bootstrap': False, 'rfc__criterion': 'entropy', 'rfc__max_depth': 6, 'rfc__n_estimators': 11, 'rfc__random_state': 12345}
Wall time: 654 ms


## Проверьте модель на тестовой выборке

Проверим ранее обученные модели на тестовой выборке, имя модели и результаты тестирования поместим в словарь model_dict для дальнейшего сравнения точностей моделей:

In [16]:
predictions_rfc = best_rfc.predict(features_test) # прогноз наилучшей модели RandomForestClassifier()
accuracy_rfc_score = accuracy_score(target_test, predictions_rfc) # Точность
print(f'Точность модели {best_rfc} на тестовой выборке составила: {round(accuracy_rfc_score*100, 2)}')

Точность модели Pipeline(steps=[('scaler', StandardScaler()),
                ('rfc',
                 RandomForestClassifier(bootstrap=False, criterion='entropy',
                                        max_depth=6, n_estimators=11,
                                        random_state=12345))]) на тестовой выборке составила: 79.45


Модель **RandomForestClassifier** показала результат 79.45 % на тестовой выборке

## (бонус) Проверьте модели на адекватность

In [17]:
model_dc = DummyClassifier(strategy='uniform', random_state=0)
model_dc.fit(features_train, target_train)
predictions_model_dc = model_dc.predict(features_test)
model_dc_score = accuracy_score(target_test, predictions_model_dc)
print(f'Точность рандомной модели на тестовой выборке составила: {round(model_dc_score*100,2)}%')

print(f'Классы на тестовой выборке модели DummyClassifier() {collections.Counter(predictions_model_dc)}')  
print(f'Классы на тестовой выборке модели RandomForestClassifier() {collections.Counter(predictions_rfc)}')  
print(f'Классы тестовой выборки {collections.Counter(target_train)}')  

Точность рандомной модели на тестовой выборке составила: 49.11%
Классы на тестовой выборке модели DummyClassifier() Counter({1: 1232, 0: 1177})
Классы на тестовой выборке модели RandomForestClassifier() Counter({0: 1988, 1: 421})
Классы тестовой выборки Counter({0: 557, 1: 246})


DummyClassifier() он же фиктивный классификатор, в нашем случае является маркером неадекватности модели, в выборке данных у нас присутсвует дисбаланс классов - это второй индикатор адекватности т.к. в прогнозе адекватной можели также должен присутствовать дисбаланс. У модели RandomForestClassifier() как раз этот дисбаланс виден в прогнозе, поэтому наша модель вполне адекватна.

**Общий вывод:** Мы ознакомились с датасетом, затем привели данные к нужным типам и округлили их. Разбили наш датасет на обучающую, валидационную и тестовую выборки. 

Мы использовали в нашем проекте три модели: **LogisticRegression**, **DecisionTreeClassifier**, **RandomForestClassifier**, применив метод перебора параметров Pipeline + GridSearchCV нам удалось достичь точность в: **74.27, 77.9, 80.5** процентов на валидационной выборке соответственно. 

Модель **RandomForestClassifier**  параметрами:

bootstrap=False 

criterion='entropy'

max_depth=6,

n_estimators=11

random_state=12345}

показала результат 79.45 % на тестовой выборке

DummyClassifier() он же фиктивный классификатор, в нашем случае является маркером неадекватности модели, в выборке данных у нас присутсвует дисбаланс классов - это второй индикатор адекватности т.к. в прогнозе адекватной можели также должен присутствовать дисбаланс. У модели RandomForestClassifier() как раз этот дисбаланс виден в прогнозе, поэтому наша модель вполне адекватна.